# Exploration of the Data Provided by Zhang et. al (2019)

## Source code classification (Open Judge)

In [12]:
## Import libraries

import pandas as pd

In [3]:
mydf = pd.read_pickle('astnn/data/programs.pkl')

In [4]:
mydf.head()

,0,1,2
0,0,"int main()\n{\n\tint a;\n\tint bai,wushi,ershi...",97
1,1,"int main()\n{\n int m,x100,x50,x20,x10,x5,x...",97
2,2,"int main()\n{\n int n,i,shuzu[111],count1=0...",97
3,3,"int main()\n{\n\tint n,a1=0,a2=0,a3=0,a4=0,a5=...",97
4,4,"int main()\n{\n\tint n,a,b,c,d,e,f;\n\ta=0;b=0...",97


In [114]:
type(mydf.iloc[:,1])

pandas.core.series.Series

In [49]:
mydf.iloc[:,0]

0            0
1            1
2            2
3            3
4            4
         ...  
51996    51996
51997    51997
51998    51998
51999    51999
52000    52000
Name: 0, Length: 52001, dtype: int64

Check the structure of single data from a single row

In [25]:
mydf.iloc[0,1]

'int main()\n{\n\tint a;\n\tint bai,wushi,ershi,shi,wu,yi;\n\tcin>>a;\n\tbai=a/100;\n\ta=a%100;\n\twushi=a/50;\n\ta=a%50;\n\tershi=a/20;\n\ta=a%20;\n\tshi=a/10;\n\ta=a%10;\n\twu=a/5;\n\ta=a%5;\n\tyi=a;\n\tcout<<bai<<endl;\n\tcout<<wushi<<endl;\n\tcout<<ershi<<endl;\n\tcout<<shi<<endl;\n\tcout<<wu<<endl;\n\tcout<<yi<<endl;\n\treturn 0;\n}'

So the data provided is one function per row.

How many labels? 

In [10]:
len(mydf.iloc[:,2].value_counts().index)

104

Distribution of data across labels:

In [11]:
mydf.iloc[:,2].value_counts()

1     501
95    500
10    500
39    500
71    500
     ... 
53    500
85    500
22    500
54    500
32    500
Name: 2, Length: 104, dtype: int64

In [14]:
mydf.iloc[:,2].value_counts().values

array([501, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500])

It is quite evenly distributed with ~500 datapoints/observations for each label

### Question: Can the software vulnerability detection dataset from Russell et al (2018) fit into the structure of this dataset?

In [78]:
val = pd.read_pickle('data/newval.pickle')

In [79]:
val.head()

,functionSource,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,combine
0,gwy_resource_class_mkdir(GwyResourceClass *kla...,False,False,False,False,False,False
1,SetIgnoredFields( const char **papszFields )\n...,False,False,False,False,False,False
2,timeoutProtoDisplays(void)\n{\n struct prot...,False,False,False,False,False,False
3,"SelectViewsOf (GraphicComp* comp, Editor* ed) ...",False,False,False,False,False,False
4,multi_get_bool(bool &value)\n{\n\tif (!Multi_s...,False,False,False,False,False,False


In [80]:
val.iloc[0,0]

'gwy_resource_class_mkdir(GwyResourceClass *klass)\n{\n    gchar *path;\n    gint ok;\n\n    g_return_val_if_fail(GWY_IS_RESOURCE_CLASS(klass), FALSE);\n\n    path = g_build_filename(gwy_get_user_dir(), klass->name, NULL);\n    if (g_file_test(path, G_FILE_TEST_IS_DIR)) {\n        g_free(path);\n        return TRUE;\n    }\n\n    ok = !g_mkdir(path, 0700);\n    g_free(path);\n\n    return ok;\n}'

* There's a difference between language.
* The vulnerability dataset is in C and C++
* The source code classification dataset is in C++ however it is clean code from OJ Bench.
* But since they're using `pycparser` as their parser, it should work for both languages

Creating the same structure for implementation on the ASTNN architecture

In [92]:
newdf = val.iloc[:,[0,6]]

In [ ]:
newdf['id'] = pd.Series(val.index.values)

In [96]:
newdf = newdf.iloc[:,[2,0,1]]

In [101]:
newdf.iloc[:,2] = newdf.iloc[:,2].map({True:0, False:1})

In [102]:
newdf

,id,functionSource,combine
0,0,gwy_resource_class_mkdir(GwyResourceClass *kla...,1
1,1,SetIgnoredFields( const char **papszFields )\n...,1
2,2,timeoutProtoDisplays(void)\n{\n struct prot...,1
3,3,"SelectViewsOf (GraphicComp* comp, Editor* ed) ...",1
4,4,multi_get_bool(bool &value)\n{\n\tif (!Multi_s...,1
...,...,...,...
127471,127471,Java_org_gdal_gdal_gdalJNI_Band_1GetHistogram_...,0
127472,127472,rb_singleton_class(VALUE obj)\n{\n VALUE kl...,1
127473,127473,"clutter_text_set_justify (ClutterText *self,\n...",1
127474,127474,set_sensitive_animation()\n{\n\tGtkWidget *rot...,1


In [187]:
from pycparser import c_parser, parse_file
import re

In [104]:
parser = c_parser.CParser()

In [190]:
k = newdf.iloc[0,1]

In [210]:
mydf.iloc[14321,1]

'int main()\n{\n\tint n,i=0,j=2,p,q,r;\n\tint m[100],t[100];\n\tscanf("%d\\n",&n);\n\tfor(i=0;i<n-1;i++)\n\t{\n\t\tscanf("%d\\n",&m[i]);\n\t}\n\tscanf("%d",&m[n-1]);\n\ti=0;\n\tfor(i=0;i<n;i++)\n\t{\n\t\tif(m[i]==1||m[i]==2)\n\t\t{\n\t\t\tt[i]=1;\n\t\t}\n\t\tif(m[i]>2)\n\t\t{\n\t\t\tp=1;\n\t\t\tq=1;\n\t\t\tfor(j=2;j<m[i];j++)\n\t\t\t{\n\t\t\t\tr=q;\n\t\t\t\tq=p+q;\n\t\t\t\tp=r;\n\t\t\t}\n\t\t\tt[i]=q;\n\t\t\tj=2;\n\t\t}\n\t}\n\ti=0;\n\tfor(i=0;i<n-1;i++)\n\t{\n\t\tprintf("%d\\n",t[i]);\n\t}\n\tprintf("%d",t[n-1]);\n\treturn 0;\n}'

In [211]:
newdf.iloc[2134,1]

'_serialize_list (GSList *objects_list, GType g_type)\n{\n  JsonArray *json_array = NULL;\n  JsonNode *list_node = NULL;\n  GObject *object = NULL;\n  GSList *item = NULL;\n  const gchar *id = NULL;\n\n  json_array = json_array_new ();\n  for (item = objects_list; item; item = g_slist_next (item))\n    {\n      /* We just serialize the ID of the group / set */\n      object = G_OBJECT (item->data);\n      if (g_type == FROGR_TYPE_PHOTOSET)\n        {\n          id = frogr_photoset_get_id (FROGR_PHOTOSET (object));\n          if (!id)\n            id = frogr_photoset_get_local_id (FROGR_PHOTOSET (object));\n        }\n      else if (g_type == FROGR_TYPE_GROUP)\n        id = frogr_group_get_id (FROGR_GROUP (object));\n\n      if (id)\n        json_array_add_string_element (json_array, id);\n    }\n\n  list_node = json_node_new(JSON_NODE_ARRAY);\n  json_node_set_array (list_node, json_array);\n\n  return list_node;\n}'

In [207]:
k = 'gwy_resource_class_mkdir(GwyResourceClass)\n{\n    char *path;\n    int ok;\n\n    g_return_val_if_fail(GWY_IS_RESOURCE_CLASS(klass), FALSE);\n\n    path = g_build_filename(gwy_get_user_dir(), klass->name, NULL);\n    if (g_file_test(path, G_FILE_TEST_IS_DIR)) {\n        g_free(path);\n        return TRUE;\n    }\n\n    ok = !g_mkdir(path, 0700);\n    g_free(path);\n\n    return ok;\n}'

In [208]:
parser.parse(k)

FileAST(ext=[FuncDef(decl=Decl(name='gwy_resource_class_mkdir',
                               quals=[
                                     ],
                               storage=[
                                       ],
                               funcspec=[
                                        ],
                               type=FuncDecl(args=ParamList(params=[ID(name='GwyResourceClass'
                                                                       )
                                                                   ]
                                                            ),
                                             type=TypeDecl(declname='gwy_resource_class_mkdir',
                                                           quals=[
                                                                 ],
                                                           type=IdentifierType(names=['int'
                                                                  

In [204]:
parse_file('test.c',use_cpp=True)

ParseError: test.c:4:10: before: ok

In [129]:
z = pd.read_pickle('astnn/data/test/test_.pkl')

In [130]:
z.head()

,id,code,label
45281,45281,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",3
40377,40377,"FileAST(ext=[Decl(name='s',\n ...",64
3220,3220,"FileAST(ext=[Decl(name='B',\n ...",90
20981,20981,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",98
36621,36621,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",26


In [131]:
z.iloc[0,1]

FileAST(ext=[FuncDef(decl=Decl(name='main',
                               quals=[
                                     ],
                               storage=[
                                       ],
                               funcspec=[
                                        ],
                               type=FuncDecl(args=None,
                                             type=TypeDecl(declname='main',
                                                           quals=[
                                                                 ],
                                                           type=IdentifierType(names=['int'
                                                                                     ]
                                                                               )
                                                           )
                                             ),
                               init=None,
                               